In [1]:
import json
import time
import requests
import numpy as np
import pandas as pd

In [65]:
# configure API request link
url_root = "https://maps.googleapis.com/maps/api/elevation"
key_str = "${YOUR_KEY}"
output_fmt = "json"
request_str = "%s/%s?locations=%s&key=%s"

In [3]:
# download data
links = pd.read_csv("./data/links_L260855.csv")

In [67]:
lat_lng_str = []
for idx, row in links.iterrows():
    startLat = row['startY']    # latitude of start node
    startLong = row['startX']   # longitude of start node
    endLat = row['endY']        # latitude of end node
    endLong = row['endX']       # longitude of start node
    lat_lng_str.append(str(startLat) + "," + str(startLong) + "|")
    lat_lng_str.append(str(endLat) + "," + str(endLong) + "|")
    
# delete duplicates
lat_lng_str = list(set(lat_lng_str))

# limit each request to 512
length = len(lat_lng_str)
iteration = length // 512 + 1

In [69]:
# !IMPORTANT!
# payment is required
for i in range(iteration):
    #print("iteration index: ", i)
    iter_str = lat_lng_str[i * 512: (i+1) * 512]
    locs = "".join([x for x in iter_str]).rstrip('|')

    # make requests
    r = requests.get(request_str % (url_root, output_fmt, locs, key_str))
    with open("./data/elevation/elevation_result_" + str(i) + ".json", "w") as json_file:
        json.dump(json.loads(r.text), json_file)
    
    # slow down API requests so we aren't blocked
    time.sleep(1)

iteration index:  0
iteration index:  1
iteration index:  2
iteration index:  3
iteration index:  4
iteration index:  5
iteration index:  6
iteration index:  7
iteration index:  8
iteration index:  9
iteration index:  10
iteration index:  11
iteration index:  12
iteration index:  13
iteration index:  14
iteration index:  15
iteration index:  16
iteration index:  17
iteration index:  18
iteration index:  19
iteration index:  20
iteration index:  21
iteration index:  22
iteration index:  23
iteration index:  24
iteration index:  25
iteration index:  26
iteration index:  27
iteration index:  28
iteration index:  29
iteration index:  30
iteration index:  31
iteration index:  32
iteration index:  33
iteration index:  34
iteration index:  35
iteration index:  36
iteration index:  37
iteration index:  38
iteration index:  39
iteration index:  40
iteration index:  41
iteration index:  42
iteration index:  43
iteration index:  44
iteration index:  45
iteration index:  46
iteration index:  47
it

In [70]:
# dictionary for saving elevation
lat_lng_elev = dict()
for i in range(iteration):
    with open("./data/elevation/elevation_result_" + str(i) + ".json", "r") as f_json:
        result = json.load(f_json)
        r = result['results']
        
        for j in range(len(r)):
            r_ = r[j]
            elevation = r_['elevation']
            loc_lat = r_['location']['lat']
            loc_lng = r_['location']['lng']
            lat_lng_elev[(loc_lat, loc_lng)] = elevation

In [71]:
# calculate elevation angle
angle = list()
for i, row in links.iterrows():
    start_lat = row['startY']
    start_lng = row['startX']
    end_lat = row['endY']
    end_lng = row['endX']
    
    start_elev = lat_lng_elev[(start_lat, start_lng)]
    end_elev = lat_lng_elev[(end_lat, end_lng)]
    
    elev_diff = end_elev - start_elev
    street_length = row['street_length']
    angle.append(np.arctan2(elev_diff, street_length))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [73]:
links['elevation'] = angle

In [9]:
links.to_csv("./data/links_L260855.csv", index=False)